In [1]:
%reload_ext autoreload
%autoreload 2

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from app.system import System
from app.tools import PropertiesFilterTool, InfoExtractorTool


In [4]:
# from app.system import Conversation, Task, Step
# from app.task_resolver import GatherBookingInfoResolver
# from app.system import System

# system_x = System()
# step = Step("GATHER_BOOKING_INFO", GatherBookingInfoResolver())
# customer_number="99513718"

# def test_step(message, customer_number="99513718"):
#     system_x.add_user_message(message, customer_number)
#     conversation = system_x.get_conversation(customer_number)
#     print(f"Conversation: {conversation}")
#     result = step.resolve(conversation)

#     print(f"Result: {result} \nStep Done: {step.is_done()}")
#     if not step.is_done():
#         system_x.add_assistant_message(result, customer_number)
#     else:
#         print("Step is Done, we can move to the next step.")

# test_step("2 personas para el sabado por 2 noches")

In [1]:
from app.system import Conversation, Task, Step
from app.task_resolver import GatherBookingInfoResolver, HouseSelectionResolver, GatherUserInfoResolver, BookingConfirmationResolver
from app.system import System
from app.task_resolver.tasks import create_make_reservation_task
import langchain

system = System()
customer_number="99513718"

# make_reservation_task = create_make_reservation_task()

def test_task(message: str, task: Task):
    
    system.add_user_message(message, customer_number)
    conversation = system.get_conversation(customer_number)
    print(f"Conversation: {conversation}")
    response = task.run(conversation)
    system.add_assistant_message(response, customer_number)
    return response
    # system_x.add_user_message(message, customer_number)
    # conversation = system_x.get_conversation(customer_number)
    # print(f"Conversation: {conversation}")
    # result = step.resolve(conversation)

    # print(f"Result: {result} \nStep Done: {step.is_done()}")
    # if not step.is_done():
    #     system_x.add_assistant_message(result, customer_number)
    # else:
    #     print("Step is Done, we can move to the next step.")

In [4]:
system_x.get_conversation(customer_number)

[{'role': 'user',
  'content': 'Hola, me gustaria reservar una casa para el sabado que viene por dos noches para 3 personas.'},
 {'role': 'assistant',
  'content': 'Hola! Tenemos dos propiedades disponibles: \n\n1. Cabaña Sol: Impresionante villa con vistas panorámicas a las montañas. Esta lujosa propiedad ofrece un ambiente tranquilo y relajante con amplios espacios interiores y exteriores. Cuenta con una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Perfecta para escapadas en familia o con amigos. Precio por noche: 250.0 USD\n\n2. Cabaña Luna: Impresionante villa con vistas panorámicas a las montañas. Esta lujosa propiedad ofrece un ambiente tranquilo y relajante con amplios espacios interiores y exteriores. Cuenta con una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Perfecta para escapadas en familia o con amigos. Precio por noche: 120.0 USD\n\nPor favor, seleccione una propiedad para 

In [3]:
# test_task("Hola, me gustaria reservar una casa para el sabado que viene por dos noches para 3 personas.")
# test_task("La mas barata")
# test_task("Agustin Sarasua, agustinsarasua@gmail.com")
test_task("Si")
# test_task("Si, me gustaria reservar la mas barata")

Conversation: [{'role': 'user', 'content': 'Hola, me gustaria reservar una casa para el sabado que viene por dos noches para 3 personas.'}, {'role': 'assistant', 'content': 'Hola! Tenemos dos propiedades disponibles: \n\n1. Cabaña Sol: Impresionante villa con vistas panorámicas a las montañas. Esta lujosa propiedad ofrece un ambiente tranquilo y relajante con amplios espacios interiores y exteriores. Cuenta con una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Perfecta para escapadas en familia o con amigos. Precio por noche: 250.0 USD\n\n2. Cabaña Luna: Impresionante villa con vistas panorámicas a las montañas. Esta lujosa propiedad ofrece un ambiente tranquilo y relajante con amplios espacios interiores y exteriores. Cuenta con una piscina privada, jardines exuberantes y una terraza para disfrutar de las maravillosas vistas. Perfecta para escapadas en familia o con amigos. Precio por noche: 120.0 USD\n\nPor favor, seleccione una propie

'La reserva ha sido realizada. Recibirá un correo electrónico con la información de la reserva y las instrucciones de pago en un plazo de 3 horas. ¡Gracias por elegir nuestro servicio!'